In [1]:
from google.colab import files
files.upload()

Saving Birth_Death_2021-11-09_10-00-00.csv to Birth_Death_2021-11-09_10-00-00.csv


{'Birth_Death_2021-11-09_10-00-00.csv': b'"year","nationality_group_en","nationality_group_ar","gender_en","gender_ar","birth_count","death_count"\n"2019","Emirati","\xd8\xa5\xd9\x85\xd8\xa7\xd8\xb1\xd8\xa7\xd8\xaa\xd9\x8a","Males","\xd8\xb0\xd9\x83\xd9\x88\xd8\xb1","3624","267"\n"2019","Emirati","\xd8\xa5\xd9\x85\xd8\xa7\xd8\xb1\xd8\xa7\xd8\xaa\xd9\x8a","Females","\xd8\xa5\xd9\x86\xd8\xa7\xd8\xab","3474","202"\n"2019","Non Emirati","\xd8\xba\xd9\x8a\xd8\xb1 \xd8\xa5\xd9\x85\xd8\xa7\xd8\xb1\xd8\xa7\xd8\xaa\xd9\x8a","Males","\xd8\xb0\xd9\x83\xd9\x88\xd8\xb1","12827","1708"\n"2019","Non Emirati","\xd8\xba\xd9\x8a\xd8\xb1 \xd8\xa5\xd9\x85\xd8\xa7\xd8\xb1\xd8\xa7\xd8\xaa\xd9\x8a","Females","\xd8\xa5\xd9\x86\xd8\xa7\xd8\xab","11941","551"\n"2018","Emirati","\xd8\xa5\xd9\x85\xd8\xa7\xd8\xb1\xd8\xa7\xd8\xaa\xd9\x8a","Males","\xd8\xb0\xd9\x83\xd9\x88\xd8\xb1","3762","244"\n"2018","Emirati","\xd8\xa5\xd9\x85\xd8\xa7\xd8\xb1\xd8\xa7\xd8\xaa\xd9\x8a","Females","\xd8\xa5\xd9\x86\xd8\xa7\xd8\xab","

In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,907 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,660 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,523 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ub

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, MinMaxScaler, VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import numpy as np

# Initialize Spark session
spark = SparkSession.builder.appName("BirthDeathPrediction").getOrCreate()

# Load the dataset
file_path = "Birth_Death_2021-11-09_10-00-00.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Encode categorical variables
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index").fit(df) for col in ["nationality_group_en", "gender_en"]]
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

# Define features and target variable
features = ["year", "nationality_group_en_index", "gender_en_index"]
target = "birth_count"

# Normalize numerical features using MinMaxScaler
assembler = VectorAssembler(inputCols=features, outputCol="features_vec")
df = assembler.transform(df)
scaler = MinMaxScaler(inputCol="features_vec", outputCol="scaled_features")
df = scaler.fit(df).transform(df)

# Prepare input (X) and target (y) variables
df = df.select(col("scaled_features").alias("features"), col(target).alias("label"))

# Split the dataset into training (80%) and testing (20%) sets
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Bagging Ensemble Implementation
num_models = 10  # Number of decision trees in the ensemble
models = []

# Train multiple decision trees on random subsets of data (bootstrap sampling)
for _ in range(num_models):
    sample_df = train_df.sample(withReplacement=True, fraction=1.0)

    # Train a Decision Tree Regressor
    dt = DecisionTreeRegressor(featuresCol="features", labelCol="label", seed=42)
    model = dt.fit(sample_df)

    # Store trained model
    models.append(model)

# Make predictions using the ensemble (average of individual models)
predictions = [model.transform(test_df).select(col("prediction").alias(f"prediction_{i}")) for i, model in enumerate(models)]
predictions_df = test_df
for i, pred in enumerate(predictions):
    predictions_df = predictions_df.join(pred, how="inner")
predictions_df = predictions_df.withColumn("avg_prediction", sum(col(f"prediction_{i}") for i in range(num_models)) / num_models)

# Display actual vs. predicted birth counts
print("Actual vs Predicted Birth Counts:")
predictions_df.select("label", "avg_prediction").show()

# 10-Fold Cross-Validation Implementation
def cross_validate(df, num_folds=10):
    fold_size = df.count() // num_folds
    errors = []

    for i in range(num_folds):
        # Define validation and training sets
        start, end = i * fold_size, (i + 1) * fold_size
        val_df = df.rdd.zipWithIndex().filter(lambda x: start <= x[1] < end).map(lambda x: x[0]).toDF()
        train_df = df.rdd.zipWithIndex().filter(lambda x: x[1] < start or x[1] >= end).map(lambda x: x[0]).toDF()

        # Train new ensemble model
        models = []
        for _ in range(num_models):
            sample_df = train_df.sample(withReplacement=True, fraction=1.0)
            dt = DecisionTreeRegressor(featuresCol="features", labelCol="label", seed=42)
            model = dt.fit(sample_df)
            models.append(model)

        # Predict and compute error (Mean Absolute Error)
        predictions = [model.transform(val_df).select(col("prediction").alias(f"prediction_{i}")) for i, model in enumerate(models)]
        predictions_df = val_df
        for i, pred in enumerate(predictions):
            predictions_df = predictions_df.join(pred, how="inner")
        predictions_df = predictions_df.withColumn("avg_prediction", sum(col(f"prediction_{i}") for i in range(num_models)) / num_models)

        evaluator = RegressionEvaluator(labelCol="label", predictionCol="avg_prediction", metricName="mae")
        error = evaluator.evaluate(predictions_df)
        errors.append(error)

    return np.mean(errors)

# Perform 10-fold cross-validation and print the mean absolute error
cv_error = cross_validate(df, num_folds=10)
print(f"\nMean Absolute Error from 10-Fold Cross-Validation: {cv_error:.2f}")

# Stop Spark session
spark.stop()

Actual vs Predicted Birth Counts:
+-----+--------------+
|label|avg_prediction|
+-----+--------------+
|10788|       11696.2|
|10788|       11696.2|
|10788|       10917.8|
|10788|       10878.2|
|10788|       11771.1|
|10788|       11696.2|
|10788|       11696.2|
|10788|       10917.8|
|10788|       10878.2|
|10788|       11771.1|
|10788|       10926.6|
|10788|       10926.6|
|10788|       10148.2|
|10788|       10108.6|
|10788|       11001.5|
|10788|       10896.0|
|10788|       10896.0|
|10788|       10117.6|
|10788|       10078.0|
|10788|       10970.9|
+-----+--------------+
only showing top 20 rows


Mean Absolute Error from 10-Fold Cross-Validation: 316.70
